# Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
    
import time
import copy

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
from cycler import cycler
color_list  = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']
marker_list = ['o','x','d','v','^','<','>']

plt.rc('axes', prop_cycle=(cycler('color',color_list)))

from LRT import LRT
from LRT import moments
from LRT import figfuns

**Parameter struct**

In [ ]:
par = LRT.ParStruct()
SAMPLE = '_p100'

## Load data

## Logs

In [ ]:
# a. load
data = LRT.dataStruct()
data.logY = np.transpose(np.genfromtxt(f'data/logY{SAMPLE}.csv',delimiter=','))

par.T, par.N = data.logY.shape
print(f'(T,N) = ({par.T},{par.N})')

# b. update par
par.simN = max([par.N,par.simN])
par.k = 5
par.k_lead = 30
par.depth = 10

## Levels

In [ ]:
par_levels = copy.deepcopy(par)

# a. load
data_levels = LRT.dataStruct()
data_levels.Y = np.transpose(np.genfromtxt(f'data/Y_zerocutoff{SAMPLE}.csv',delimiter=','))

# remove anynone with some negative 
I = np.min(data_levels.Y, axis=0) >= 0
data_levels.Y = data_levels.Y[:,I]

par_levels.T, par_levels.N = data_levels.Y.shape
print(f'(T,N) = ({par_levels.T},{par_levels.N})')

I = data_levels.Y == 0
print(f'share of zero observations: {I.mean()*100:.2f}')
print(f'share of individuals with at least one zero observation: {np.max(I,axis=0).mean()*100:.2f}')

# b. update par
par_levels.simN = max([par_levels.N,par_levels.simN])

# c. hack to get LRT.estimate to work
data_levels.logY = np.copy(data_levels.Y)

# Estimate

## Logs

In [ ]:
model_logs = LRT.estimate(par, data, 'LRT (depth 10)')
model_logs.name_short = 'LRT_depth_10'
model_logs.color = color_list[0]
model_logs.marker = marker_list[0]

## Levels

In [ ]:
model_levels = LRT.estimate(par_levels, data_levels, 'LRT (depth 10, levels)')
model_levels.name_short = 'LRT_depth_10_levels'
model_levels.color = color_list[1]
model_levels.marker = marker_list[1]

# Simulate

In [ ]:
model_logs.data = LRT.simulate(model_logs.par,model_logs,data)
model_logs.data.Y = np.exp(model_logs.data.logY)

model_levels.data = LRT.simulate(model_levels.par,model_levels,data_levels)
model_levels.data.Y = copy.deepcopy(model_levels.data.logY)

# Figures

In [ ]:
models = [model_levels,model_logs]

for diff in [False, True]: 
    for mom in ['mean','var','skew','kurt']: 
        figfuns.robust_age_profile(data_levels,models,mom,diff,prefix='levels',save_to_disk=True)